# 多输入多输出通道

In [2]:
import torch
from d2l import torch as d2l

#### 1. 多通道输入互相关运算

In [3]:
def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [13]:
# 验证
X = torch.tensor([
    [[0, 1, 2], [3, 4, 5], [6, 7, 8]],
    [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
])

K = torch.tensor([
    [[0, 1], [2, 3]],
    [[1, 2], [3, 4]]
])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

#### 2. 多通道输出互相关运算

In [14]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [15]:
print(K)
K = torch.stack((K, K + 1, K + 2), 0)
print(K.shape)
print(K)

tensor([[[0, 1],
         [2, 3]],

        [[1, 2],
         [3, 4]]])
torch.Size([3, 2, 2, 2])
tensor([[[[0, 1],
          [2, 3]],

         [[1, 2],
          [3, 4]]],


        [[[1, 2],
          [3, 4]],

         [[2, 3],
          [4, 5]]],


        [[[2, 3],
          [4, 5]],

         [[3, 4],
          [5, 6]]]])


In [8]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

#### 3. 1×1的卷积等价于全连接

In [16]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [17]:
X = torch.normal(0,1,(3,3,3))
K = torch.normal(0,1,(2,3,1,1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1-Y2).sum())<1e-6